In [1]:
import zipfile
import json
import csv
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sys
import os


In [2]:
energy_list = ['COAL', 'NATURAL_GAS', 'ELECTRICITY_NET_GENERATION', 
               'PETROLEUM_RESIDENTIAL_COMMERCIAL', 'PETROLEUM_INDUSTRIAL', 'PETROLEUM_TRANSPORTATION_ELECTRICPOWER']

for i in energy_list:
    os.system("sed 's/Not Available/0/g' {}.csv > {}_clean.csv".format(i, i))



In [3]:
#clean data

def cleandata(data, prefix):
    remove_total = []

    with open (data) as f:
        for line in f:
            if (line.strip().split(',')[1][-3] + line.strip().split(',')[1][-2]) != '13' and line.strip().split(',')[4] == prefix and line.strip().split(',')[5] == ' Total"':
                remove_total.append(line.strip().split(','))
    
    for i in remove_total:
        #print(i[2].split('"')[1].split('"')[0])
        i[2] = float(i[2].split('"')[1].split('"')[0])

    remove_total.insert(0, ['MSN','YYYYMM','Value','Column_Order','Description',  'Total', 'Unit'])  
 
    with open (data.split('.')[0] + '_total_consumption.csv', 'w') as f:
        wr = csv.writer(f)
        wr.writerows(remove_total)
    
    
def cleandata_electrity(data):   
    remove_total = []
    with open (data) as f:
        for line in f:
            #print(line.strip().split(','))
            if (line.strip().split(',')[1][-3] + line.strip().split(',')[1][-2]) != '13' and line.strip().split(',')[4] == '"Electricity Net Generation Total (including from sources not shown)':
                remove_total.append(line.strip().split(','))
                
    for i in remove_total:
        #print(i[2].split('"')[1].split('"')[0])
        i[2] = float(i[2].split('"')[1].split('"')[0])
                
    remove_total.insert(0, ['MSN','YYYYMM','Value','Column_Order','Description', 'All_sectors', 'Unit'])  
    
    with open (data.split('.')[0] + '_total_consumption.csv', 'w') as f:
        wr = csv.writer(f)
        wr.writerows(remove_total)
        

'''        
with open ('PETROLEUM_INDUSTRIAL_clean.csv') as f:
    for line in f:
        #print(line.strip().split(','))
        #if (line.strip().split(',')[1][-3] + line.strip().split(',')[1][-2]) != '13' and line.strip().split(',')[4] == "Total Petroleum Consumed by the Industrial Sector":
        print(line.strip().split(',')[4])
'''        
        
        
def cleandata_petro(data, prefix):   
    remove_total = []
    with open (data) as f:
        for line in f:
            #print(line.strip().split(','))
            if (line.strip().split(',')[1][-3] + line.strip().split(',')[1][-2]) != '13' and line.strip().split(',')[4] == prefix:
                remove_total.append(line.strip().split(','))
                
    for i in remove_total:
        #print(i[2].split('"')[1].split('"')[0])
        i[2] = float(i[2].split('"')[1].split('"')[0])
                
    remove_total.insert(0, ['MSN','YYYYMM','Value','Column_Order','Description',  'Unit'])  
    
    with open (data.split('.')[0] + '_total_consumption.csv', 'w') as f:
        wr = csv.writer(f)
        wr.writerows(remove_total)
        

        
cleandata('COAL_clean.csv', '"Coal Consumption')
cleandata('NATURAL_GAS_clean.csv', '"Natural Gas Consumption')
cleandata_electrity('ELECTRICITY_NET_GENERATION_clean.csv')

cleandata_petro('PETROLEUM_INDUSTRIAL_clean.csv', '"Total Petroleum Consumed by the Industrial Sector"')
cleandata_petro('PETROLEUM_RESIDENTIAL_COMMERCIAL_clean.csv', '"Total Petroleum Consumed by the Commercial Sector"')
cleandata_petro('PETROLEUM_TRANSPORTATION_ELECTRICPOWER_clean.csv', '"Total Petroleum Consumed by the Electric Power Sector"')



In [4]:
total_co2 = pd.read_csv('Carbon_Dioxide_Emissions_From_Energy.csv')
COAL = pd.read_csv('COAL_clean_total_consumption.csv')
NATURALGAS = pd.read_csv('NATURAL_GAS_clean_total_consumption.csv')
ELECTRICITY = pd.read_csv('ELECTRICITY_NET_GENERATION_clean_total_consumption.csv')
PETRO_INDUSTRIAL = pd.read_csv('PETROLEUM_INDUSTRIAL_clean_total_consumption.csv')
PETRO_RESIDENTIAL_COMMERCIAL = pd.read_csv('PETROLEUM_RESIDENTIAL_COMMERCIAL_clean_total_consumption.csv')
PETRO_TRANSPORTATION_ELECTRICPOWER = pd.read_csv('PETROLEUM_TRANSPORTATION_ELECTRICPOWER_clean_total_consumption.csv')


Xcoal = COAL['Value'] # "Thousand Short Tons"
Xnatural = NATURALGAS['Value'] # "Billion Cubic Feet"
Xelectrity = ELECTRICITY['Value'] # "Million Kilowatthours"
Xpetro_indus = PETRO_INDUSTRIAL['Value'] # "Thousand Barrels per Day"
Xpetro_res_com = PETRO_RESIDENTIAL_COMMERCIAL['Value'] # "Thousand Barrels per Day"
Xpetro_trans_elec = PETRO_TRANSPORTATION_ELECTRICPOWER['Value'] # "Thousand Barrels per Day"

Yco2 = total_co2['Total Energy CO2 Emissions (Million Metric Tons of Carbon Dioxide)']

year = []
for i in range(1973, 2023):
    for j in range(1, 13):
        if j < 10:
            year.append(str(i) + '0' + str(j))
        else:
            year.append(str(i) + str(j))
            

data_variable = pd.DataFrame({
                   'Year': year,
                   'COAL, Thousand Short Tons': Xcoal.values.tolist(),
                   'NATURALGAS, Billion Cubic Feet': Xnatural.values.tolist(),
                   'ELECTRICITY, Million Kilowatthours': Xelectrity.values.tolist(),
                   'PETRO_INDUSTRIAL, Thousand Barrels per Day': Xpetro_indus.values.tolist(),
                   'PETRO_RESIDENTIAL_COMMERCIAL, Thousand Barrels per Day': Xpetro_res_com.values.tolist(),
                   'PETRO_TRANSPORTATION_ELECTRICPOWER, Thousand Barrels per Day': Xpetro_trans_elec.values.tolist(),
                   'CO2, Million Metric Tons': Yco2.values.tolist()
                  })


data_variable['COAL, Thousand Short Tons'] = data_variable['COAL, Thousand Short Tons'].astype(float)
data_variable['NATURALGAS, Billion Cubic Feet'] = data_variable['NATURALGAS, Billion Cubic Feet'].astype(float)
data_variable['ELECTRICITY, Million Kilowatthours'] = data_variable['ELECTRICITY, Million Kilowatthours'].astype(float)
data_variable['PETRO_INDUSTRIAL, Thousand Barrels per Day'] = data_variable['PETRO_INDUSTRIAL, Thousand Barrels per Day'].astype(float)
data_variable['PETRO_RESIDENTIAL_COMMERCIAL, Thousand Barrels per Day'] = data_variable['PETRO_RESIDENTIAL_COMMERCIAL, Thousand Barrels per Day'].astype(float)
data_variable['PETRO_TRANSPORTATION_ELECTRICPOWER, Thousand Barrels per Day'] = data_variable['PETRO_TRANSPORTATION_ELECTRICPOWER, Thousand Barrels per Day'].astype(float)
data_variable['CO2, Million Metric Tons'] = data_variable['CO2, Million Metric Tons'].astype(float)

# write the modified dataframe back to a CSV file
data_variable.to_csv('ml_data_variable.csv', index=False)
